In [35]:
# -*- coding:utf-8 -*-
import pymongo
import locachoice_utils

def get_monggo_db():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn

def get_result_post_addr_geo():
    db = get_monggo_db()
    return db['result_post_addr_geo']

def get_src_post_info_clean_price():
    db = get_monggo_db()
    return db['tmp_post_info_clean_price']

def get_result_grids():
    conn = get_monggo_db()
    return conn['result_grids']
        
def tidy_result_post_info_clean_price():
    db = get_monggo_db()
    src_coll = db['result_post_info_clean_price']
    dst_coll = db['tmp_post_info_clean_price']
    cursor = src_coll.find()
    for cur in cursor:
        cur['coaddr'] = cur.get('coaddr').strip()
        salary = cur.get('postaveragesalary').strip()
        if salary == '':
            continue
        cur['postaveragesalary'] = int(salary)
        dst_coll.save(cur)        


def append_gridid2result_post_addr_geo():
    coll = get_result_post_addr_geo()
    cursor = coll.find()
    for cur in cursor:
        gridid = locachoice_utils.get_grid_id(cur.get('lng'), cur.get('lat'))
        print(gridid)
        coll.update_one({'_id': cur.get('_id')}, {'$set': {'gridid': gridid}})     

def append_avgsalary2result_post_addr_geo():
    src_coll = get_src_post_info_clean_price()
    coll = get_result_post_addr_geo()
    cursor = coll.find({'gridid': {'$ne': -1}, 'avgsalary': {'$exists': False}})
    for cur in cursor:
        print('procing one ...')
        coaddr = cur.get('coaddr')
        t_salary = 0
        count = 0
        docs = src_coll.find({'coaddr': coaddr})
        for doc in docs:
            count += 1
            t_salary += doc.get('postaveragesalary')
            
        if count == 0:
            print('error addr.')
            coll.delete_one({'_id': cur.get('_id')})
            continue
            
        avgsalary = t_salary // count
        coll.update_one({'_id': cur.get('_id')}, {'$set': {'avgsalary': avgsalary}})
        
def export2result_grid():
    src_coll = get_result_post_addr_geo()
    dst_coll = get_result_grids()
    gridids = dst_coll.distinct('gridid')
    for gridid in gridids:
        print('procing one gridid ...')
        sala = None
        t_salary = 0
        count = 0
        cursor = src_coll.find({'gridid': gridid})
        for cur in cursor:
            count += 1
            t_salary += cur.get('avgsalary')
        
        print(count)
        if count == 0:
            sala =None
        else:
            sala = t_salary // count
        print(t_salary, sala)
        dst_coll.update_one({'gridid': gridid}, {'$set': {'avgsalary': sala}})

def get_coaddrs(gridid):
    coll = get_result_post_addr_geo()
    cursor = coll.find({'gridid': gridid})
    addrs = []
    for cur in cursor:
        addrs.append(cur.get('coaddr'))
    return addrs
        
def export_types_avgsalary2grid():
    src_coll = get_src_post_info_clean_price()
    dst_coll = get_result_grids()
    gfzqy =['股份制企业', '上市公司']
    wsdz =['外商独资']
    gq = ['国企', '事业单位', '国家机关']
    my = ['民营']
    hz = ['合资']
    gridids = dst_coll.distinct('gridid')
    for gridid in gridids:
        print('procing one gridid ...')
        gfzqy_total = 0
        wsdz_total = 0
        gq_total = 0
        my_total = 0
        hz_total = 0          
        gfzqy_count = 0
        wsdz_count = 0
        gq_count = 0
        my_count = 0
        hz_count = 0       
        addrs = get_coaddrs(gridid)        
        for addr in addrs:
            cursor = src_coll.find({'coaddr': addr})
            for cur in cursor:
                conature = cur.get('conature')
                if conature in gfzqy:
                    gfzqy_count += 1
                    gfzqy_total += cur.get('postaveragesalary')
                elif conature in wsdz:
                    wsdz_count += 1
                    wsdz_total += cur.get('postaveragesalary')
                elif conature in gq:
                    gq_count += 1
                    gq_total += cur.get('postaveragesalary')
                elif conature in my:
                    my_count += 1
                    my_total += cur.get('postaveragesalary')
                elif conature in hz:
                    hz_count += 1
                    hz_total += cur.get('postaveragesalary')
                else:
                    pass
        
        if gfzqy_count == 0:
            gfzqy_avgsalary = None
        else:
            gfzqy_avgsalary = gfzqy_total // gfzqy_count
        if wsdz_count == 0:
            wsdz_avgsalary = None
        else:
            wsdz_avgsalary = wsdz_total // wsdz_count
        if gq_count == 0:
            gq_avgsalary = None
        else:
            gq_avgsalary = gq_total // gq_count            
        if my_count == 0:
            my_avgsalary = None
        else:
            my_avgsalary = my_total // my_count
        if hz_count == 0:
            hz_avgsalary = None
        else:
            hz_avgsalary = hz_total // hz_count            
        
        print(gfzqy_count, wsdz_count, gq_count, my_count, hz_count)
        toset_doc = {
            'gfzqy_avgsalary': gfzqy_avgsalary,
            'wsdz_avgsalary': wsdz_avgsalary,
            'gq_avgsalary': gq_avgsalary,
            'my_avgsalary': my_avgsalary,
            'hz_avgsalary': hz_avgsalary,            
        }
        print(toset_doc)
        dst_coll.update_one({'gridid': gridid}, {'$set': toset_doc})

if __name__ == '__main__':
    print('starting')
#     export_types_avgsalary2grid()    
    print('ended ended')


starting
procing one gridid ...
0 0 0 14 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 3900, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 979 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 6132, 'hz_avgsalary': None}
procing one gridid ...
0 58 35 684 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': 4250, 'gq_avgsalary': 6185, 'my_avgsalary': 5827, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avg

0 0 0 215 37
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 5841, 'hz_avgsalary': 5000}
procing one gridid ...
115 0 0 762 256
{'gfzqy_avgsalary': 7300, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 7978, 'hz_avgsalary': 5109}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 57 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 5000, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary'

0 0 0 47 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 7127, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
54 315 0 472 0
{'gfzqy_avgsalary': 4462, 'wsdz_avgsalary': 1000, 'gq_avgsalary': None, 'my_avgsalary': 5331, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 154 15
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 4038, 'hz_avgsalary': 6066}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary':

0 28 0 100 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': 4428, 'gq_avgsalary': None, 'my_avgsalary': 6822, 'hz_avgsalary': None}
procing one gridid ...
347 0 0 0 0
{'gfzqy_avgsalary': 9142, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
2343 0 0 210 77
{'gfzqy_avgsalary': 10200, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 6285, 'hz_avgsalary': 3603}
procing one gridid ...
60 0 0 508 0
{'gfzqy_avgsalary': 12191, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 4281, 'hz_avgsalary': None}
procing one gridid ...
619 180 91 147 0
{'gfzqy_avgsalary': 8616, 'wsdz_avgsalary': 6350, 'gq_avgsalary': 7000, 'my_avgsalary': 7591, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, '

0 0 0 51 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 5088, 'hz_avgsalary': None}
procing one gridid ...
8 0 0 65 0
{'gfzqy_avgsalary': 4500, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 4807, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': None, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 138 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 4257, 'hz_avgsalary': None}
procing one gridid ...
8 0 0 1039 13
{'gfzqy_avgsalary': 5500, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 6865, 'hz_avgsalary': 7884}
procing one gridid ...
0 0 0 47 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 8276, 'hz_avgsalary': None}
procing one gridid ...
0 0 0 0 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary':

2303 1799 931 10479 1536
{'gfzqy_avgsalary': 8700, 'wsdz_avgsalary': 9503, 'gq_avgsalary': 8201, 'my_avgsalary': 12898, 'hz_avgsalary': 15300}
procing one gridid ...
972 51 456 4217 335
{'gfzqy_avgsalary': 7645, 'wsdz_avgsalary': 14176, 'gq_avgsalary': 10712, 'my_avgsalary': 14884, 'hz_avgsalary': 7560}
procing one gridid ...
322 5 0 1361 39
{'gfzqy_avgsalary': 20076, 'wsdz_avgsalary': 5000, 'gq_avgsalary': None, 'my_avgsalary': 11067, 'hz_avgsalary': 6192}
procing one gridid ...
468 4 4 2857 6
{'gfzqy_avgsalary': 7699, 'wsdz_avgsalary': 6500, 'gq_avgsalary': 6375, 'my_avgsalary': 8102, 'hz_avgsalary': 4333}
procing one gridid ...
252 98 285 1376 28
{'gfzqy_avgsalary': 7298, 'wsdz_avgsalary': 7897, 'gq_avgsalary': 10572, 'my_avgsalary': 6967, 'hz_avgsalary': 7410}
procing one gridid ...
0 0 6 435 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': 15500, 'my_avgsalary': 5566, 'hz_avgsalary': None}
procing one gridid ...
1674 78 63 3735 325
{'gfzqy_avgsalary': 9063, 'wsd

0 0 0 46 0
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 5260, 'hz_avgsalary': None}
procing one gridid ...
17 0 36 1014 223
{'gfzqy_avgsalary': 5000, 'wsdz_avgsalary': None, 'gq_avgsalary': 5000, 'my_avgsalary': 6888, 'hz_avgsalary': 5000}
procing one gridid ...
66 0 48 218 0
{'gfzqy_avgsalary': 4962, 'wsdz_avgsalary': None, 'gq_avgsalary': 4451, 'my_avgsalary': 4169, 'hz_avgsalary': None}
procing one gridid ...
0 132 0 861 497
{'gfzqy_avgsalary': None, 'wsdz_avgsalary': 9397, 'gq_avgsalary': None, 'my_avgsalary': 6181, 'hz_avgsalary': 8266}
procing one gridid ...
14 0 0 49 0
{'gfzqy_avgsalary': 3000, 'wsdz_avgsalary': None, 'gq_avgsalary': None, 'my_avgsalary': 1500, 'hz_avgsalary': None}
procing one gridid ...
518 0 39 185 0
{'gfzqy_avgsalary': 6100, 'wsdz_avgsalary': None, 'gq_avgsalary': 5000, 'my_avgsalary': 3583, 'hz_avgsalary': None}
procing one gridid ...
149 138 65 2693 400
{'gfzqy_avgsalary': 8803, 'wsdz_avgsalary': 7347, 'gq_avgsala

In [43]:
# -*- coding:utf-8 -*-
import pymongo
import locachoice_utils

def get_monggo_db():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn

def get_result_grids():
    conn = get_monggo_db()
    return conn['result_grids']

coll = get_result_grids()
# count = 0
# total = 0
# docs = coll.find({'avgsalary': {'$ne': None}})
# for doc in docs:
#     count += 1
#     total += doc.get('avgsalary')
    
# print(count, total, total // count)
# dir(coll)
# help(coll.delete_many)
db = get_monggo_db()
# dir(db)
help(db.get_collection)

Help on method get_collection in module pymongo.database:

get_collection(name, codec_options=None, read_preference=None, write_concern=None, read_concern=None) method of pymongo.database.Database instance
    Get a :class:`~pymongo.collection.Collection` with the given name
    and options.
    
    Useful for creating a :class:`~pymongo.collection.Collection` with
    different codec options, read preference, and/or write concern from
    this :class:`Database`.
    
      >>> db.read_preference
      Primary()
      >>> coll1 = db.test
      >>> coll1.read_preference
      Primary()
      >>> from pymongo import ReadPreference
      >>> coll2 = db.get_collection(
      ...     'test', read_preference=ReadPreference.SECONDARY)
      >>> coll2.read_preference
      Secondary(tag_sets=None)
    
    :Parameters:
      - `name`: The name of the collection - a string.
      - `codec_options` (optional): An instance of
        :class:`~bson.codec_options.CodecOptions`. If ``None`` (the
  